In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, isnan, sum, when, isnull, regexp_replace
# Importa os tipos de dados necessários para o cast correto
from pyspark.sql.types import LongType, IntegerType, DoubleType

# Configurações do Spark (MANTIDAS)
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

spark = SparkSession.builder.config(conf=conf).appName("UnificarUPA").getOrCreate()

# ======================
# ARQUIVOS E CAMINHOS
# ======================
s3_prefix = 's3a://bucket-raw-upa-connect-sofh/arquivos/'

arquivos_sensor = [
    '2025_10_13.csv',
    '2025_10_12.csv',
    '2025_10_11.csv',
    '2025_10_10.csv',
    '2025_10_09.csv'
]


# ----------------------------------------------------
# ALTERAÇÃO: 'fk_paciente' REMOVIDA da lista de colunas finais
# ----------------------------------------------------
colunas_finais = ['data_hora', 'valor', 'fk_sensor', 'fk_unid_medida', 'fk_upa']  


# ======================
# LEITURA DOS ARQUIVOS
# ======================

sensor_paths = [s3_prefix + f for f in arquivos_sensor]


sensor_df = spark.read.option('header', 'true').option('delimiter', ',').csv(sensor_paths)
# Leitura do atendimento_df omitida ou ajustada
atendimento_df = spark.createDataFrame([], schema=sensor_df.schema)


# Adicionar colunas que não existam em algum dos DataFrames
for col_name in colunas_finais:
    if col_name not in sensor_df.columns:
        sensor_df = sensor_df.withColumn(col_name, lit(None))
    if col_name not in atendimento_df.columns:
        atendimento_df = atendimento_df.withColumn(col_name, lit(None))

# Selecionar e ordenar as colunas
sensor_df = sensor_df.select(colunas_finais)


# ==========================================
# TRATAMENTO DE OUTLIERS E NULOS (REVISADA)
# ==========================================

print(f"Total de linhas antes do tratamento: {sensor_df.count()}")

# 1. Criar uma coluna temporária 'valor_limpo'
#    - Primeiro, substitui vírgula (,) por ponto (.) para padronização
#    - Segundo, remove todos os caracteres que NÃO sejam dígitos (0-9) ou o ponto (.)
sensor_df = sensor_df.withColumn(
    "valor_limpo",
    regexp_replace(
        regexp_replace(col("valor"), ",", "."), # 1. Substitui vírgula por ponto
        "[^0-9\\.]", ""                         # 2. Remove lixo restante (seu regex)
    )
)

# 1.1. Fazer o cast condicional, substituindo a coluna 'valor' original
#    - se fk_sensor == 1, converte para Long (inteiro, sem decimais)
#    - se fk_sensor == 2 (ou qualquer outro valor), converte para Double (mantém decimais)
sensor_df = sensor_df.withColumn(
    "valor", # Substitui a coluna 'valor' original
    when(col("fk_sensor") == 1,
         col("valor_limpo").cast(LongType()))
    .otherwise(
         col("valor_limpo").cast(DoubleType()))
).drop("valor_limpo") # Remove a coluna temporária que não é mais necessária


# 1.2 Cast das outras colunas
sensor_df = sensor_df.withColumn("fk_sensor", col("fk_sensor").cast(IntegerType()))
sensor_df = sensor_df.withColumn("fk_unid_medida", col("fk_unid_medida").cast(IntegerType()))

# --- Condições de EXCLUSÃO ---
# NOTA: O cast para LongType (e não double) garante que números grandes inteiros sejam comparados corretamente.

# 1. fk_sensor = 1 (Geral): apagar linhas em que valor é MAIOR que 300, e MENOR que 0
# CÂMERA VISÃO COMPUTACIONAL
condicao_exclusao_1 = (
    (col("fk_sensor") == 1) &
    (
        (col("valor") > 300) |               # Maior que 300
        (col("valor") < 0)                   # Menor que 0
    )
)

# 2. fk_sensor = 2 e fk_unid_medida = 1: apagar linhas em que valor é MAIOR que 37 e MENOR que 12
# TEMPERATURA DO AMBIENTE
condicao_exclusao_2 = (
    (col("fk_sensor") == 2) &
    (col("fk_unid_medida") == 1) &
    (
        (col("valor") > 37) |              
        (col("valor") < 12)
    )
)

# 3. fk_sensor = 2 e fk_unid_medida = 2: apagar linhas em que valor é MAIOR que 95 e MENOR que 10
# UMIDADE DO AMBIENTE
condicao_exclusao_3 = (
    (col("fk_sensor") == 2) &
    (col("fk_unid_medida") == 2) &
    (
        (col("valor") > 95) |              
        (col("valor") < 10)
    )
)

# Combinação das condições: Uma linha deve ser APAGADA se atender a QUALQUER uma das condições de OUTLIER
condicao_outlier_total = condicao_exclusao_1 | condicao_exclusao_2 | condicao_exclusao_3

# Condição final de exclusão (Outlier OU valor que se tornou NULL/NaN)
# Se você quer APAGAR os outliers E os valores que falharam no cast (NULL/NaN), use a lógica abaixo:
condicao_exclusao_total = (
    condicao_outlier_total         # Exclui outliers numéricos (agora precisos)
    | col("valor").isNull()        # Exclui valores estritamente NULL (inclui falha de cast)
    | isnan(col("valor"))          # Exclui NaN (menos comum com LongType, mas boa prática)
)

# Aplicar o filtro: Manter apenas as linhas onde a condição de exclusão (outlier ou nulo) NÃO é verdadeira.
sensor_df = sensor_df.filter(~condicao_exclusao_total)

# ======================
# RESULTADOS
# ======================
sensor_df.show(10)
print(f"Total de linhas após o tratamento: {sensor_df.count()}")

# Opcionalmente, pode descomentar a seção abaixo se quiser o cálculo de nulos, mas
# a contagem de nulos na coluna 'valor' agora será zero, pois eles foram excluídos.

# # ==========================================
# # CONTAGEM ESPECÍFICA PARA NULL NA COLUNA 'VALOR'
# # ==========================================

# # 1. Cria a expressão de agregação: soma 1 se o valor for estritamente NULL.
# total_nulos_valor = sum(
#      when(col("valor").isNull(), 1).otherwise(0)
# ).alias("total_nulos_estritos_valor")

# # 2. Aplica a agregação e coleta o resultado
# resultado = sensor_df.agg(total_nulos_valor).collect()

# # 3. Exibe o resultado de forma limpa
# contagem = resultado[0]["total_nulos_estritos_valor"]

# print(f"\n=========================================================")
# print(f"Contagem de registros com valor NULL na coluna 'valor': {contagem}")
# print(f"=========================================================")

# # Você também pode simplesmente mostrar o DataFrame resultante da agregação:
# sensor_df.agg(total_nulos_valor).show()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-88a48f3b-859c-4b34-8c17-9da6bedcfa97;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 445ms :: artifacts dl 23ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	---------------------------------------------------------------------
	|     

Total de linhas antes do tratamento: 293755
+-------------------+-----+---------+--------------+------+
|          data_hora|valor|fk_sensor|fk_unid_medida|fk_upa|
+-------------------+-----+---------+--------------+------+
|2025-10-13T00:05:00| 17.0|        1|          null|     1|
|2025-10-13T00:10:00| 15.0|        1|          null|     1|
|2025-10-13T00:15:00| 12.0|        1|          null|     1|
|2025-10-13T00:20:00| 12.0|        1|          null|     1|
|2025-10-13T00:25:00| 10.0|        1|          null|     1|
|2025-10-13T00:35:00| 79.0|        1|          null|     1|
|2025-10-13T00:40:00|278.0|        1|          null|     1|
|2025-10-13T00:45:00| 12.0|        1|          null|     1|
|2025-10-13T00:50:00| 13.0|        1|          null|     1|
|2025-10-13T00:55:00| 10.0|        1|          null|     1|
+-------------------+-----+---------+--------------+------+
only showing top 10 rows



Total de linhas após o tratamento: 254693


In [2]:
FINAL_OUTPUT_DIR = "s3a://bucket-trusted-upa-connect-sofh/"
FINAL_FILENAME = "tabela_sensores_tratada.csv"
TEMP_STAGING_DIR = f"{FINAL_OUTPUT_DIR}/_temp_staging_integrated"

# 1. Escreve o resultado no caminho temporário
print(f"\nEscrevendo dados temporariamente em: {TEMP_STAGING_DIR}")

# NOTA: Coalesce(1) para garantir a geração de um único arquivo CSV.
sensor_df.coalesce(1).write \
    .option('delimiter', ';') \
    .option('header', 'true') \
    .option('encoding', 'UTF-8') \
    .mode('overwrite') \
    .csv(TEMP_STAGING_DIR)

# 2. Renomeia o arquivo gerado
try:
    # Acessa a classe 'Path' da JVM através do gateway do Spark
    Path = spark._jvm.org.apache.hadoop.fs.Path
    
    # Acessa a configuração do Hadoop
    hadoop_conf = spark._jsc.hadoopConfiguration()
    
    # Obtém o objeto FileSystem para o caminho temporário
    fs = Path(TEMP_STAGING_DIR).getFileSystem(hadoop_conf)

    # Encontra o arquivo gerado (part-00000-*.csv) dentro do diretório temporário
    list_status = fs.globStatus(Path(TEMP_STAGING_DIR + "/part-00000-*.csv"))

    if list_status:
        # Pega o caminho completo do arquivo gerado
        generated_file_path = list_status[0].getPath()

        # Define o caminho final e o nome específico para o arquivo
        final_output_path = Path(f"{FINAL_OUTPUT_DIR}/{FINAL_FILENAME}")

        # Renomeia (move) o arquivo para o caminho e nome definitivos
        fs.rename(generated_file_path, final_output_path)
        
        # 3. Deleta o diretório temporário (que ficou vazio) e outros arquivos de metadados
        fs.delete(Path(TEMP_STAGING_DIR), True) 
        
        print(f"\n✅ Base integrada salva e renomeada com sucesso para: {final_output_path}")

    else:
        print("\nErro: Não foi possível encontrar o arquivo CSV gerado (part-00000-*.csv) no caminho temporário.")

except Exception as e:
    print(f"\nOcorreu um erro durante a renomeação do arquivo no S3: {e}")

# Encerra a sessão Spark
spark.stop()


Escrevendo dados temporariamente em: s3a://bucket-trusted-upa-connect-sofh//_temp_staging_integrated


25/10/19 21:37:26 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/10/19 21:37:27 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.



✅ Base integrada salva e renomeada com sucesso para: s3a://bucket-trusted-upa-connect-sofh/tabela_sensores_tratada.csv
